# ワークフロー実行結果を書き戻す

このノートブックでは、変更内容をGINリポジトリに書き戻します。  
※各タスクのセルでpapermillによって自動実行されるノートブックなので手作業での実施は非推奨です。

In [ ]:
SIBLING_NAME = 'gin'
SAVE_MESSAGE = '[GIN] modify on binder'
PATH = '/home/jovyan'
IS_RECURSIVE = True
TO_GIT = False
RESULT_RENDERER = 'default'
UNLOCK = True
REMOVE_PUSH_PATH = False

## 1.変更内容を書き戻す

In [ ]:
from datalad import api
import traceback
import requests
import json
import os

DATALAD_MESSAGE = ''
DATALAD_ERROR = ''
CONNECT_REPO_ERROR = 'リポジトリに接続できません。リポジトリが存在しているか確認してください。'
CONFLICT_ERROR = 'リポジトリ側の変更と競合しました。競合を解決してください。'
PUSH_ERROR = 'リポジトリへの同期に失敗しました。'
SUCCESS = 'データ同期が完了しました。次の処理にお進みください。'

def push(DATALAD_MESSAGE, DATALAD_ERROR):
    # git annex unlock & push
    try:
        if UNLOCK:
            api.unlock(PATH)
        else:
            pass
        if REMOVE_PUSH_PATH:
            api.push(to=SIBLING_NAME, result_renderer=RESULT_RENDERER, recursive=IS_RECURSIVE)
        else:
            api.push(to=SIBLING_NAME, result_renderer=RESULT_RENDERER, path=PATH, recursive=IS_RECURSIVE)
    except:
        DATALAD_ERROR = traceback.format_exc()
        DATALAD_MESSAGE = PUSH_ERROR
    else:
        DATALAD_MESSAGE = SUCCESS
    finally:
        return DATALAD_MESSAGE, DATALAD_ERROR


In [ ]:
from datalad import api
import traceback
import os

# add & commit & pull
try:
    api.save(message=SAVE_MESSAGE, path=PATH, recursive=IS_RECURSIVE, to_git=TO_GIT)
    api.update(path=PATH, sibling=SIBLING_NAME, how='merge')
except:
    DATALAD_ERROR = traceback.format_exc()
    # リモートへの接続エラー
    if 'Repository does not exist:' in DATALAD_ERROR:
        try:
            # リモートリポジトリのURLを最新化する
            os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
            from util.scripts import utils
            utils.update_repo_url()
            os.chdir(os.environ['HOME'])
        except:
            # リポジトリ自体が無いときなど
            DATALAD_MESSAGE = CONNECT_REPO_ERROR
        else:
            # エラーを解消し、再度、同期処理を行う
            DATALAD_ERROR = ''
            try:
                api.update(path=PATH, sibling=SIBLING_NAME, how='merge')
            except:
                DATALAD_ERROR = traceback.format_exc()
                DATALAD_MESSAGE = CONFLICT_ERROR
            else:
                DATALAD_MESSAGE, DATALAD_ERROR = push(DATALAD_MESSAGE, DATALAD_ERROR)
    else:
        DATALAD_MESSAGE = CONFLICT_ERROR
else:
    DATALAD_MESSAGE, DATALAD_ERROR = push(DATALAD_MESSAGE, DATALAD_ERROR)
finally:
    # 実行結果を格納する
    %store DATALAD_MESSAGE
    %store DATALAD_ERROR